In [5]:
#include <memory>
#include <iostream>
#include <map>
#include <string>
#include <atomic>

> When you create dynamic resources you need to think about ownership ie. who is responsible for deleting it? The object that should be responsible for deleting the resource should manage the resource using some kind of smart pointer (or a container).

Unlike std::unique_ptr, which is designed to singly own and manage a resource, std::shared_ptr is meant to solve the case where you need multiple smart pointers co-owning a resource.
    
This means that it is fine to have multiple std::shared_ptr pointing to the same resource. Internally, std::shared_ptr keeps track of how many std::shared_ptr are sharing the resource. As long as at least one std::shared_ptr is pointing to the resource, the resource will not be deallocated, even if individual std::shared_ptr are destroyed. As soon as the last std::shared_ptr managing the resource goes out of scope (or is reassigned to point at something else), the resource will be deallocated.

* 1 Basic operation of std::shared_ptr
* 2 re-implenent std::shared_ptr
* 3 what happens when std::shared_ptr in a function declration
* 4 what happens when std::shared_ptr as member variable in a class

In [6]:
class Resource
{
public:
    int number;
    
    Resource(int n) {
        number = n;
        std::cout << "Resource acquired\n"; 
    }
    
    ~Resource() { 
        std::cout << "Resource destroyed\n"; 
    }
    
    friend std::ostream& operator<<(std::ostream& out, const Resource& res){
        out << "I am a resource\n";
        return out;
    }
};

# 1 Basic operation of std::shared_ptr

In [7]:
{
    // allocate a Resource object and have it owned by std::shared_ptr
    Resource* res { new Resource(111) };
    std::shared_ptr<Resource> ptr1{ res };
    //returns the number of shared_ptr objects referring to the same managed object
    std::cout << "the number of shared_ptr objects referring to the same managed object: " << ptr1.use_count() << std::endl; 
    {
        std::shared_ptr<Resource> ptr2 { ptr1 }; // make another std::shared_ptr pointing to the same thing
        std::cout << "the number of shared_ptr objects referring to the same managed object: " << ptr1.use_count() << std::endl;
        std::cout << "Killing one shared pointer\n";
    } // ptr2 goes out of scope here, but nothing happens
    
    std::cout << "the number of shared_ptr objects referring to the same managed object: " << ptr1.use_count() << std::endl;
    std::cout << "Killing another shared pointer\n";
} // ptr1 goes out of scope here, and the allocated Resource is destroyed

Resource acquired
the number of shared_ptr objects referring to the same managed object: 1
the number of shared_ptr objects referring to the same managed object: 2
Killing one shared pointer
the number of shared_ptr objects referring to the same managed object: 1
Killing another shared pointer
Resource destroyed


In [14]:
{
    std::shared_ptr<Resource> sptr1( new Resource(123) );
    std::shared_ptr<Resource> sptr2 = sptr1;
    std::shared_ptr<Resource> sptr3;
    std::shared_ptr<Resource> sptr4;
 
    sptr3 = sptr2;

    std::cout << sptr1.use_count() << std::endl;
    std::cout << sptr2.use_count() << std::endl;
    std::cout << sptr3.use_count() << std::endl;

    sptr4 = sptr2;

    std::cout << sptr1.use_count() << std::endl;
    std::cout << sptr2.use_count() << std::endl;
    std::cout << sptr3.use_count() << std::endl;
}


Resource acquired
3
3
3
4
4
4
Resource destroyed


In [17]:
{
    std::shared_ptr<Resource> sptr1( new Resource(123) );
    std::shared_ptr<Resource> sptr2 = sptr1;
    std::shared_ptr<Resource> sptr3;
    std::shared_ptr<Resource> sptr4;
 
    sptr3 = sptr2;

    std::cout << sptr1.use_count() << std::endl;
    std::cout << sptr2.use_count() << std::endl;
    std::cout << sptr3.use_count() << std::endl;

    sptr4 = std::move(sptr2);
    
    std::cout << sptr1.use_count() << std::endl;
    std::cout << sptr2.use_count() << std::endl;
    std::cout << sptr3.use_count() << std::endl;
    std::cout << sptr4.use_count() << std::endl;
    if(sptr2.get() == nullptr){
        std::cout << "std::move set the pointer in sptr2 to nullptr" << std::endl;
    }
    
}

Resource acquired
3
3
3
3
0
3
3
std::move set the pointer in sptr2 to nullptr
Resource destroyed


# 2 re-implement std::shared_ptr

Unlike std::unique_ptr, which uses a single pointer internally, std::shared_ptr uses two pointers internally. One pointer points at the resource being managed. The other points at a “control block”, which is a dynamically allocated object that tracks of a bunch of stuff, including how many std::shared_ptr are pointing at the resource. When a std::shared_ptr is created via a std::shared_ptr constructor, the memory for the managed object (which is usually passed in) and control block (which the constructor creates) are allocated separately.

In [8]:
class SharedCount {
   private:
    std::atomic<int> count_;

    
   public:
    SharedCount() : count_{1} {}

    void add() { ++count_; }

    void minus() { --count_; }

    int get() const { return count_; }
};

In [9]:
template <typename T>
class SharedPtr {
    private:
    T* ptr_;
    SharedCount* ref_count_;

    
   public:
    SharedPtr(T* ptr) : ptr_{ptr}, ref_count_{new SharedCount} {}

    SharedPtr() : ptr_{nullptr}, ref_count_{new SharedCount} {}

    ~SharedPtr() { clean(); }
    
    //copy constructor
    SharedPtr(const SharedPtr& p) {
        this->ptr_ = p.ptr_;
        this->ref_count_ = p.ref_count_;
        ref_count_->add();
    }
    
    // move constructor
    SharedPtr(SharedPtr&& p) {
        this->ptr_ = p.ptr_;
        this->ref_count_ = p.ref_count_;
        p.ptr_ = nullptr;
        p.ref_count_ = nullptr;
    }
    
    // copy assignment operator
    SharedPtr& operator=(const SharedPtr& p) {
        if (this == &p) return *this;
        clean();
        this->ptr_ = p.ptr_;
        this->ref_count_ = p.ref_count_;
        ref_count_->add();
        return *this;
    }
    
    // move assignment operator
    SharedPtr& operator=(SharedPtr&& p) {
        if (this == &p) return *this;
        clean();
        this->ptr_ = p.ptr_;
        this->ref_count_ = p.ref_count_;
        p.ptr_ = nullptr;
        p.ref_count_ = nullptr;
        return *this;
    }

    int use_count() { return ref_count_->get(); }

    T* get() const { return ptr_; }

    T* operator->() const { return ptr_; }

    T& operator*() const { return *ptr_; }

    operator bool() const { return ptr_; }
    
    private:
    void clean() {
        if (ref_count_) {
            ref_count_->minus();
            if (ref_count_->get() == 0) {
                if (ptr_) delete ptr_;
                delete ref_count_;
            }
        }
    }

};

In [10]:
{
    // allocate a Resource object and have it owned by std::shared_ptr
    Resource* res { new Resource(222) };
    SharedPtr<Resource> ptr1{ res };
    //returns the number of shared_ptr objects referring to the same managed object
    std::cout << "the number of shared_ptr objects referring to the same managed object: " << ptr1.use_count() << std::endl; 
    {
        SharedPtr<Resource> ptr2 { ptr1 }; // make another std::shared_ptr pointing to the same thing
        std::cout << "the number of shared_ptr objects referring to the same managed object: " << ptr1.use_count() << std::endl;
        std::cout << "Killing one shared pointer\n";
    } // ptr2 goes out of scope here, but nothing happens
    
    std::cout << "the number of shared_ptr objects referring to the same managed object: " << ptr1.use_count() << std::endl;
    std::cout << "Killing another shared pointer\n";
} // ptr1 goes out of scope here, and the allocated Resource is destroyed

Resource acquired
the number of shared_ptr objects referring to the same managed object: 1
the number of shared_ptr objects referring to the same managed object: 2
Killing one shared pointer
the number of shared_ptr objects referring to the same managed object: 1
Killing another shared pointer
Resource destroyed


This also explains why independently creating two std::shared_ptr pointed to the same resource gets us into trouble. Each std::shared_ptr will have one pointer pointing at the resource. However, each std::shared_ptr will independently allocate its own control block, which will indicate that it is the only pointer owning that resource. Thus, when that std::shared_ptr goes out of scope, it will deallocate the resource, not realizing there are other std::shared_ptr also trying to manage that resource.

However, when a std::shared_ptr is cloned using copy assignment, the data in the control block can be appropriately updated to indicate that there are now additional std::shared_ptr co-managing the resource.

# 3 when to how to use smart pointer as function argument?

* question 1: whether or not to use a smart pointer as function argument?
* question 2: Should we pass a shared_ptr by reference or by value?

I think people are unnecessarily scared of using raw pointers as function parameters. If the function is not going to store the pointer or otherwise affect its lifetime, a raw pointer works just as well and represents the lowest common denominator. Consider for example how you would pass a unique_ptr into a function that takes a shared_ptr as a parameter, either by value or by const reference?
~~~
void DoSomething(myClass * p);

DoSomething(myClass_shared_ptr.get());
DoSomething(myClass_unique_ptr.get());
~~~
A raw pointer as a function parameter does not prevent you from using smart pointers in the calling code, where it really matters.

* Guideline: Don’t pass a smart pointer as a function parameter unless you want to use or manipulate the smart pointer itself, such as to share or transfer ownership.

* Guideline: Express that a function will store and share ownership of a heap object using a by-value shared_ptr parameter.

* Guideline: Use a non-const shared_ptr& parameter only to modify the shared_ptr. Use a const shared_ptr& as a parameter only if you’re not sure whether or not you’ll take a copy and share ownership; otherwise use widget* instead (or if not nullable, a widget&).

# 4 forbidden behavior in std::shared_ptr

## double deallocation of heap-object

~~~
{
    Resource* res { new Resource(111) };
    std::shared_ptr<Resource> ptr1 { res };
    {
        std::shared_ptr<Resource> ptr2 { res }; // create ptr2 directly from res (instead of ptr1)

        std::cout << "Killing one shared pointer\n";
    } // ptr2 goes out of scope here, and the allocated Resource is destroyed

    std::cout << "Killing another shared pointer\n";
} // ptr1 goes out of scope here, and the allocated Resource is destroyed again
~~~
The difference here is that we created two std::shared_ptr independently from each other. As a consequence, even though they’re both pointing to the same Resource, they aren’t aware of each other. When ptr2 goes out of scope, it thinks it’s the only owner of the Resource, and deallocates it. When ptr1 later goes out of the scope, it thinks the same thing, and tries to delete the Resource again. Then bad things happen.

Fortunately, this is easily avoided: if you need more than one std::shared_ptr to a given resource, copy an existing std::shared_ptr.